In [10]:
from binance import *
from ta import *
from backtest import *

import matplotlib.pyplot as plt

In [11]:
symbol = 'SOL/USDT'
freq = '1h'

In [12]:
plot_prices([symbol[:-5]], freq)

In [13]:
dict_df = get_dict_df(symbols=[symbol], freq=freq, columns=['close', 'open'])

dict_df = add_dict_df_emas(dict_df, lengths=[12, 21])

df = dict_df[symbol]

df

,close,open,EMA_12,EMA_21,Signal_EMA_12_21,Delta_EMA_12_21,Rel_Delta_EMA_12_21,Ratio_EMA_12_21
timestamp,,,,,,,,
2023-01-01 00:00:00,9.99,9.97,NaN,NaN,0,NaN,NaN,NaN
2023-01-01 01:00:00,10.04,9.99,NaN,NaN,0,NaN,NaN,NaN
2023-01-01 02:00:00,10.02,10.04,NaN,NaN,0,NaN,NaN,NaN
2023-01-01 03:00:00,9.97,10.02,NaN,NaN,0,NaN,NaN,NaN
2023-01-01 04:00:00,9.84,9.97,NaN,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2024-09-29 19:00:00,159.42,157.98,157.467558,157.202326,0,0.265232,0.001687,0.168720
2024-09-29 20:00:00,160.24,159.43,157.894088,157.478478,0,0.415610,0.002639,0.263915
2024-09-29 21:00:00,160.07,160.23,158.228844,157.714071,0,0.514772,0.003264,0.326396


In [14]:
# df_plot = df[['close', 'Signal_EMA_12_21']]
# # df_plot = df[['EMA_12', 'EMA_21']]
df_plot = df[['Ratio_EMA_12_21']]

plot_df(df_plot)

In [15]:
trades_prev, trades_next, trades_2prev, trades_3prev = get_trades(df, 'Signal_EMA_12_21', freq)

In [16]:
# buy and hold
entry = df['close'].iloc[0]
exit = df['close'].iloc[-1]

f'Return: {round((exit/entry-1)*100,2)} %'

'Return: 1486.19 %'

In [19]:
value_list = []
compound_value_list = []
range_list = [x/100 for x in range(1, 50)]
for pct in range_list:
    df_rel_delta = df[['open', 'Ratio_EMA_12_21']].copy()
    df_rel_delta['Signal_Ratio_EMA_12_21'] = np.where((df_rel_delta['Ratio_EMA_12_21'] < pct) & (df_rel_delta['Ratio_EMA_12_21'].shift(1) > pct), -1,
                                            np.where((df_rel_delta['Ratio_EMA_12_21'] > -pct) & (df_rel_delta['Ratio_EMA_12_21'].shift(1) < -pct), 1, 0))
    df_rel_delta.drop(columns='Ratio_EMA_12_21', inplace=True)
    _, trades_next_rel_delta, _, _ = get_trades(df_rel_delta, 'Signal_Ratio_EMA_12_21', freq)

    value, df_pnl = pnl(trades_next_rel_delta, verbose=False)
    value_list.append(value)

    compound_value, df_compound_pnl, df_compound_return = pnl_compund(trades_next_rel_delta, verbose=False)
    compound_value_list.append(compound_value)

df_roi = pd.DataFrame({'pct': range_list, 'value': value_list})
df_roi.set_index('pct', inplace=True)
plot_df(df_roi)
# plot_df(df_pnl)

df_compound_roi = pd.DataFrame({'pct': range_list, 'compound_value': compound_value_list})
df_compound_roi.set_index('pct', inplace=True)
plot_df(df_compound_roi)
# plot_df(df_compound_pnl)
# plot_df(df_compound_return*100)